In [153]:
import requests

In [165]:
base = 'https://www.olympedia.org/countries/USA/editions/'

In [155]:
page = [61,59,54,53,26,25,24,23,22,21]

In [166]:
for n in page:
    print(base + str(n))

https://www.olympedia.org/countries/USA/editions/61
https://www.olympedia.org/countries/USA/editions/59
https://www.olympedia.org/countries/USA/editions/54
https://www.olympedia.org/countries/USA/editions/53
https://www.olympedia.org/countries/USA/editions/26
https://www.olympedia.org/countries/USA/editions/25
https://www.olympedia.org/countries/USA/editions/24
https://www.olympedia.org/countries/USA/editions/23
https://www.olympedia.org/countries/USA/editions/22
https://www.olympedia.org/countries/USA/editions/21


In [156]:
html = "https://www.olympedia.org/countries/USA/editions/61"
raw_html = requests.get(html).content
from bs4 import BeautifulSoup
soup_doc = BeautifulSoup(raw_html, "html.parser")
# soup_doc.prettify()

data = soup_doc.find_all('td')
result = []
current_list = []

for item in data:
    if item.find('h2'):
        if current_list:
            result.append(current_list)
        current_list = [item]
    else:
        current_list.append(item)


if current_list:
    result.append(current_list)

r = []
for sport in result:
    for s in sport:
        if s.find('h2') or s.find('a') or s.find('span'):
            r.append(s)

# AI toools

I used ChatGPT to generate next step which helped me save a lot of time.
https://chatgpt.com/share/67a421c2-fd4c-8001-ae1a-ca1b88a1316e

I copied one section of my r and give this prompt to ChatGPT: 

In this context:  

- `h2` represents the sports category.  
- `href="results"` corresponds to gender, usually appearing in the format `"xxx,[gender]"`.  
- If `results` is not **individual**, then all participants should be placed in a single cell.  
- `href="athletes"` corresponds to the athlete's name.  
- If a `<span>` appears after `athlete`, it indicates a **medal**; if no `<span>` appears, it means **no medal**.

In [157]:
rows = []
current_sport = None

i = 0
while i < len(r):
    td = r[i]
    
    if td.find('h2'):
        current_sport = td.get_text(strip=True)
        i += 1
        continue
    
    # event and gender
    if td.find('a') and 'results' in td.a['href']:
        event_gender = td.get_text(strip=True)
        event, gender = event_gender.rsplit(',', 1)
        i += 1
        
        athletes = []
        medals = []
        
        # name
        while i < len(r) and (r[i].find('a') and 'athletes' in r[i].a['href'] or r[i].find('span')):
            if r[i].find('a'):
                athlete_name = r[i].get_text(strip=True)
                medal = r[i+1].get_text(strip=True) if i+1 < len(r) and r[i+1].find('span') else 'None'
                athletes.append((athlete_name, medal))
            i += 1
        
        # team as a group
        if "Team" in event:
            athlete_names = ', '.join([a[0] for a in athletes])
            medals_str = ', '.join(set(a[1] for a in athletes if a[1] != 'None')) or 'None'
            rows.append([current_sport, event.strip(), gender.strip(), athlete_names, medals_str])
        else:
            # individual
            for athlete_name, medal in athletes:
                rows.append([current_sport, event.strip(), gender.strip(), athlete_name, medal])
    else:
        i += 1


df = pd.DataFrame(rows, columns=['Sport', 'Event', 'Gender', 'Athlete', 'Medal'])

In [161]:
# add a new column that is the year
y = soup_doc.find('h1').text
import re
year = ''.join(re.findall(r'\d+', y))
df['Year']=[year for i in range(len(df))]
df.to_csv(f"{year}.csv",index=False)

In [159]:
df.head()

,Sport,Event,Gender,Athlete,Medal,Year
0,3x3 Basketball,3x3 Basketball,Women,Kelsey Plum•Jackie Young•Stefanie Dolson•Allis...,None,2020
1,Archery,Individual,Men,Brady Ellison,None,2020
2,Archery,Individual,Men,Jacob Wukie,None,2020
3,Archery,Individual,Men,Jack Williams,None,2020
4,Archery,Team,Men,Brady Ellison•Jack Williams•Jacob Wukie,None,2020


loop through 1984 - 2020

In [187]:
df_dict={}
for n in page:
    html = base + str(n)
    raw_html = requests.get(html).content
    from bs4 import BeautifulSoup
    soup_doc = BeautifulSoup(raw_html, "html.parser")
    # soup_doc.prettify()
    
    data = soup_doc.find_all('td')
    
    # add a new column that is the year
    y = soup_doc.find('h1').text
    import re
    year = ''.join(re.findall(r'\d+', y))

    # find all tds
    result = []
    current_list = []
    
    for item in data:
        if item.find('h2'):
            if current_list:
                result.append(current_list)
            current_list = [item]
        else:
            current_list.append(item)
    
    
    if current_list:
        result.append(current_list)
    
    r = []
    for sport in result:
        for s in sport:
            if s.find('h2') or s.find('a') or s.find('span'):
                r.append(s)

    rows = []
    current_sport = None
    
    i = 0
    while i < len(r):
        td = r[i]
        
        if td.find('h2'):
            current_sport = td.get_text(strip=True)
            i += 1
            continue
        
        # event and gender
        if td.find('a') and 'results' in td.a['href']:
            event_gender = td.get_text(strip=True)
            event, gender = event_gender.rsplit(',', 1)
            i += 1
            
            athletes = []
            medals = []
            
            # name
            while i < len(r) and (r[i].find('a') and 'athletes' in r[i].a['href'] or r[i].find('span')):
                if r[i].find('a'):
                    athlete_name = r[i].get_text(strip=True)
                    medal = r[i+1].get_text(strip=True) if i+1 < len(r) and r[i+1].find('span') else 'None'
                    athletes.append((athlete_name, medal))
                i += 1
            
            # team as a group
            if "Team" in event:
                athlete_names = ', '.join([a[0] for a in athletes])
                medals_str = ', '.join(set(a[1] for a in athletes if a[1] != 'None')) or 'None'
                rows.append([current_sport, event.strip(), gender.strip(), athlete_names, medals_str])
            else:
                # individual
                for athlete_name, medal in athletes:
                    rows.append([current_sport, event.strip(), gender.strip(), athlete_name, medal])
        else:
            i += 1


    df_dict[f'df{year}']= pd.DataFrame(rows, columns=['Sport', 'Event', 'Gender', 'Athlete', 'Medal'])
    df_dict[f'df{year}']['Year']=[year for i in range(len(df_dict[f'df{year}']))]


In [188]:
frames = pd.concat(df_dict.values())

In [189]:
frames.to_csv('total.csv',index = False)

In [190]:
frames.describe()

,Sport,Event,Gender,Athlete,Medal,Year
count,4746,4746,4746,4746,4746,4746
unique,52,208,4,2995,4,10
top,Athletics,Individual,Men,Blaine Wilson,None,1996
freq,1227,241,2563,20,3906,507


# I mannually added the Paris 2024 results (with medals only )

In [193]:
paris = pd.read_csv('2024.csv')

In [195]:
new = pd.concat([frames,paris])

In [196]:
new.describe()

,Sport,Event,Gender,Athlete,Medal,Year
count,4976,4976,4976,4976,4976,4976
unique,61,308,4,3099,5,12
top,Athletics,Individual,Men,Blaine Wilson,None,1996
freq,1277,241,2654,20,3906,507


In [197]:
new.to_csv('Final.csv',index = False)